In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt

# 读取数据

In [ ]:
# 本函数已保存在d2lzh_pytorch包中方便以后使用
def load_data_fashion_mnist(batch_size, resize=None, root='/home/kesci/input/FashionMNIST2065'):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())
    
    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2)

    return train_iter, test_iter

In [4]:
def xyplot(x_val, y_val, name):
    plt.figure(figsize =(3,3))
    plt.plot(x_val().detach.numpy(), y_val,detach().numpy())
    plt.xlabel('x')
    plt.ylabel(name+"x")

In [6]:
class Flatten(nn.Module):
    def forward(self,x):
        return x.view(x.shape[0],-1)

class Reshape(nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)

class LeNet(nn.Module):
    def __init__(self):
        super(self,LeNet).__init__()
        self.conv2d = nn.Sequential(
            Reshape(),
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), #b*1*28*28  =>b*6*28*28
            nn.Sigmoid(),                                                       
            nn.AvgPool2d(kernel_size=2, stride=2),                              #b*6*28*28  =>b*6*14*14
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),           #b*6*14*14  =>b*16*10*10
            nn.Sigmoid(),
            nn.AvgPool2d(kernel_size=2, stride=2),                              #b*16*10*10  => b*16*5*5
            Flatten() 
        )
        self.dense = nn.Sequential(
                nn.Linear(in_features=16*5*5, out_features=120),
                nn.Sigmoid(),
                nn.Linear(120, 84),
                nn.Sigmoid(),
                nn.Linear(84, 10)
         )
    def forward(x):
        y = self.dense(self.conv2d(x))
        return y

In [7]:
def train_ch5(net, train_iter, test_iter,criterion, num_epochs, batch_size, device,lr=None):
    net.to(device)
    optimizer = optim.SGD(net.parameters(),lr= lr)
    for i in num_epochs:
        for x,y in train_iter:
            net.train()
            optimizer.zero_grad()
            x = x.to(device)
            y = y.to(device)
            l = criterion(net(x),y)
            l.backward()
            optimizer.step()

# 初始化参数

In [ ]:
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

net.apply(init_weights)
net = net.to(device)